In [68]:
import openml
import pandas as pd
import plotly.express as px 
from sklearn.utils import shuffle
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold 

In [34]:
# List all datasets and their properties
openml.datasets.list_datasets(output_format="dataframe")

# Get dataset by ID
dataset = openml.datasets.get_dataset(1476)

# Get the data itself as a dataframe (or otherwise)
X, _, _, _ = dataset.get_data(dataset_format="dataframe")

Answer to question 1:

This archive contains 13910 measurements from 16 chemical sensors utilized in simulations for drift compensation in a discrimination task of 6 gases at various levels of concentrations.
The goal is to achieve good performance (or as low degradation as possible) over time, as reported in the paper mentioned below in Section 2: Data collection.
The primary purpose of providing this dataset is to make it freely accessible online to the chemo-sensor research community and artificial intelligence to develop strategies to cope with sensor/concept drift. The dataset can be used exclusively for research purposes. Commercial purposes are fully excluded.
The dataset was gathered within January 2007 to February 2011 (36 months) in a gas delivery platform facility situated at the ChemoSignals Laboratory in the BioCircuits Institute, University of California San Diego.
Being completely operated by a fully computerized environment controlled by a LabVIEW's National Instruments software on a PC fitted with the appropriate serial data acquisition boards. The measurement system platform provides versatility for obtaining the desired concentrations of the chemical substances of interest with high accuracy and in a highly reproducible manner, minimizing thereby the common mistakes caused by human intervention and making it possible to exclusively concentrate on the chemical sensors for compensating real drift.
The resulting dataset comprises recordings from six distinct pure gaseous substances, namely Ammonia, Acetaldehyde, Acetone, Ethylene, Ethanol, and Toluene, each dosed at a wide variety of concentration values ranging from 5 to 1000 ppmv

Answer to question 2:

There was no need to make any changes in order to import the dataset. 

Answer to question 3:



In [35]:
# Preparation of the Dataset 
X = X.astype('int64')
y = X['Class']
X = X.drop(columns=['Class'])
X,y = shuffle(X,y,random_state=0)
X = X.reset_index(drop= True)
y = y.reset_index(drop= True)

#There are no null values
X.isna().sum().sum()
y.isna().sum()


0

In [41]:
# Train - Test split

Xtrain = X[0:int(0.7*len(X))]
Xtest = X[int(0.7*len(X))+1:]
ytrain = y[0:int(0.7*len(X))]
ytest = y[int(0.7*len(X))+1:]

In [62]:
# Dummy Classifier

dummy_clf = DummyClassifier(strategy="uniform")
dummy_clf.fit(Xtrain, ytrain)

# Mean Accuracy


# F1 Score 
f1Score = f1_score(ytest, dummy_clf.predict(Xtest), average='macro')

In [91]:
# Dummy Classifier

dummy_clf = DummyClassifier(strategy="uniform")
acc_score = []
f1_scores = []
kf = KFold(n_splits=10, random_state=None)

for train_index , test_index in kf.split(Xtrain):
    X_train , X_test = X.iloc[train_index],X.iloc[test_index]
    y_train , y_test = y[train_index] , y[test_index]
     
    dummy_clf.fit(X_train, y_train)
    pred_values = dummy_clf.predict(X_test)
     
    accuracy = dummy_clf.score(pred_values, y_test)
    acc_score.append(accuracy)

    f1Score = f1_score(y_test, dummy_clf.predict(X_test), average='macro')
    f1_scores.append(f1Score)
     
avg_acc_score = sum(acc_score)/10